### Maze environment
Check out `maze.jl` to see the custom types and functions that define the belief_maze environment. 

In [120]:
include("maze.jl")
Random.seed!(2)
environment, start = generate_deterministic_environment(6)
print_maze(environment, start)


 #   #   .   .   .   .  

 .   .   G   .   #   #  

 .   .   .   #   X   #  

 .   .   #   #   .   .  

 .   .   .   .   .   .  

 .   .   .   .   .   .  



#### Reward function
We define the reward function to be -1 for every move in which the agent does not reach the goal.

In [77]:
function get_reward(pos::Pos, maze::Maze)::Int
    if maze[pos] == goal
        return 100
    else
        return -1
    end
end;

# Thompson sampling for the maze problem

Consists of two tasks:
1. a running model of our beliefs about the nature of the problem that becomes more refined as we interact with it.
2. a way to find the optimal policy for each fixed value of our beliefs.


In each step, we sample from the belief distribution, find the optimal policy for that belief, and execute the policy to get a reward. We then update our belief distribution based on the reward received.


(1) In this simple maze problem, what needs to be captured by the belief distribution are the maze and the controller. We do not know what the maze looks like, i.e. where obstacles and the goal are. We do not know the mapping between directions and buttons, so we need to learn that too.


(2) Given a maze and a mapping between directions and buttons, we can find an optimal policy using different algorithms. Below are implementations of value iteration, policy iteration and Q-learning for the maze problem. These algorithms are used to find the optimal policy for a fixed belief distribution (controller mapping).

### Thompson sampling
The agent doesn't know the environment. The environment variable is only used for simulating the agent's interaction with it, but it cannot be used to find the optimal policy. The agent needs to learn the environment by interacting with it.

##### From TS tutorial: 
TS can be applied fruitfully to a broad array of online decision problems beyond the Bernoulli bandit, and we now consider a more general setting. Suppose the agent applies a sequence of actions x1, x2, x3, . . . to a system, selecting each from a set X . This action set could be finite, as in the case of the Bernoulli bandit, or infinite. After applying action xt, the agent observes an outcome yt, which the system randomly generates according to a conditional probability measure qθ(·|xt). The agent enjoys a reward rt = r(yt), where r is a known function. The agent is initially uncertain about the value of θ and represents his uncertainty using a prior distribution p.

- action set X: [a, b, c, d]
- outcome set Y: Pos(n,n)
- probability measure qθ(·|xt): the conditional probability of reaching a position y given action x, given by the environment (maze, current position, controller, movement probabilities).
- reward function r(yt): -1 for every move in which the agent does not reach the goal.
- prior distribution p: belief distribution over the maze and controller.

##### Conditioning
- draw a random sample from p
- apply action that maximizes expected reward for the model
  - expected reward given an action under the current belief: ${\mathbb{E}_{\hat{\theta}}}[r(y_t)|x_t = x] = \sum_{o} q_{\hat{\theta}}(o|x) r(o)$
- update p by conditioning on the realized observation ${\hat{y}_t}$
  - conditional distribution: ${\mathbb{P}_{p,q}(\theta = u | x_t,y_t) = \frac{p(u)q_u(y_t|x_t)}{\sum_{v} p(v)q_v(y_t|x_t)}}$

It is important to run a whole episode until a goal is reached, otherwise the reward of an action is too insignificant.




## Finding an optimal policy
### Value iteration

In [152]:
include("value_iteration.jl");


In [153]:
optimal_policy, state_values = value_iteration(environment.maze, environment.controller, 0.9, 0.01)
for pos in keys(state_values)
    state_values[pos] = round(Int, state_values[pos])  # Use round or Int based on your needs
end
print_maze(environment, start, optimal_policy, )
print_maze(environment, start, optimal_policy, state_values)



 #↓   #↓   .↓   .↓   .←   .←  

 .→   .→   G↑   .←   #←   #↑  

 .↑   .↑   .↑   #↑   X↓   #↓  

 .↑   .↑   #↑   #↓   .↓   .↓  

 .↑   .↑   .←   .←   .←   .←  

 .↑   .↑   .↑   .↑   .↑   .↑  

 #↓0.0  #↓0.0  .↓100.0  .↓89.0  .←79.0  .←70.0 

 .→89.0  .→100.0  G↑0.0  .←100.0  #←0.0  #↑0.0 

 .↑79.0  .↑89.0  .↑100.0  #↑0.0  X↓37.0  #↓0.0 

 .↑70.0  .↑79.0  #↑0.0  #↓0.0  .↓43.0  .↓37.0 

 .↑62.0  .↑70.0  .←62.0  .←55.0  .←48.0  .←43.0 

 .↑55.0  .↑62.0  .↑55.0  .↑48.0  .↑43.0  .↑37.0 



## Learning belief models

In [80]:
# Gen Functions
using Gen

@dist function labeled_categorical(labels, probs)
    index = categorical(probs)
    labels[index]
end;

@gen function simulate_action(maze::Maze, controller::Controller, pos::Pos, button::Button)::Pos
    probs = controller[button]
    possible_targets = [get_new_pos(pos, maze, dir) for dir in DIRECTIONS]
    target = {button => :new_pos} ~ labeled_categorical(possible_targets, [probs[1], probs[2], probs[3], probs[4]])
    return target
end;

@gen function simulate_episode(maze::Maze, controller::Controller, start::Pos, episode_length::Int, policy::Policy)
    maze = environment.maze
    pos = start
    playing = true
    visited = [pos]
    rewards = []
    for t in 1:episode_length
        button = policy[pos]
        new_pos = {t => pos} ~ simulate_action(maze, controller, pos, button)
        if playing 
            pos = new_pos
            push!(visited, pos)
            push!(rewards, get_reward(pos, maze))
            if maze[pos] == goal
                playing = false
                break
            end
        end
    end
    return Episode(rewards, visited)
end;


In [81]:
trace = simulate(simulate_episode, (environment.maze, environment.controller, start, 100, optimal_policy))
choices = get_choices(trace)
print_choices(environment, trace)

 # # . . . .
 . . G . # #
 . . . # . #
 . . # # . .
 . . . . . .
 . . . . . .


In [82]:
@gen function select_controller()::Controller
    controller = Controller()
    for action in [a,b,c,d]
        probs = {action} ~ dirichlet([1.0, 1.0, 1.0, 1.0])
        controller[action] = (probs[1], probs[2], probs[3], probs[4])
    end
    return controller
end;

In [83]:
select_controller()

Dict{Button, NTuple{4, Float64}} with 4 entries:
  b => (0.054375, 0.175113, 0.500057, 0.270455)
  a => (0.287185, 0.1225, 0.566458, 0.0238573)
  c => (0.134206, 0.275522, 0.513829, 0.0764424)
  d => (0.469313, 0.0525681, 0.0280788, 0.45004)

In [84]:
# this model is not very good because it only has very few parameters and 
# it will be hard to learn the maze precisely since the categorical distributions leave a lot to chance.

@gen function select_maze(n::Int)::Maze
    maze = Maze()
    goal_x = {:goal_x} ~ uniform_discrete(1, n)
    goal_y = {:goal_y} ~ uniform_discrete(1, n)
    frac_obstacles = {:frac_obstacles} ~ beta(1,1)
    for x in 1:n
        for y in 1:n
            labels = [obstacle, empty, goal]
            pos = Pos(x, y)
            if x == goal_x && y == goal_y
                maze[pos] = {pos} ~ labeled_categorical(labels, [0.0001, 0.0001, 0.9998])
            else
                maze[pos] = {pos} ~ labeled_categorical(labels, [frac_obstacles, 1-frac_obstacles-0.0001, 0.0001])
            end
        end
    end
    return maze
end;

In [85]:
maze = select_maze(6)
print_maze(maze, Pos(1,1))

 # # # # # #
 # # # # . #
 # G . # # #
 . # . . # .
 # . # # . #
 X # # # # #


In [86]:
Random.seed!(2)
environment, start = generate_deterministic_environment(6)
print_maze(environment.maze, start)

 # # . . . .
 . . G . # #
 . . . # X #
 . . # # . .
 . . . . . .
 . . . . . .
